# Freckle Data Engineer Challenge
*22 Oct 2017  
Farooq Qaiser* 

## Admin  

initialize PySpark

In [2]:
import findspark

findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Freckle_challenge") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Load some of the basic libraries (we'll load others as we need them). 

In [5]:
from pyspark.sql import functions as func

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Set some display options. 

In [6]:
# show plots inline
%matplotlib inline

Set some parameters.  

In [12]:
seed = 1

input_path = "/home/fqaiser94/Data Engineer Challenge/location-data-sample/*.gz"

Read in data as dataframe

In [15]:
df = spark.read.json(input_path)

## RDD vs Dataframe

**RE: The expectation for this exercise is that you use Spark 2.x with Scala, Python, or Java. You can use the RDD or Dataframe APIs as you see fit, but please be ready to explain your choices.**  


I chose to use the Dataframe API over RDD API because:  
1. Dataframe API is able to take advantage of Spark’s Catalyst optimizer by exposing expressions and data fields to a query planner.
2. Dataframe API has speed advantage in most cases (see [here](http://www.adsquare.com/comparing-performance-of-spark-dataframes-api-to-spark-rdd/)). 
3. I find Dataframes an easier construction to work with  

## EDA

Always helpful to do some EDA to understand our data before diving in.  
Lets take a look at the dataframes schema.  

In [17]:
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- api_key: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- beacon_major: long (nullable = true)
 |-- beacon_minor: long (nullable = true)
 |-- beacon_uuid: string (nullable = true)
 |-- city: string (nullable = true)
 |-- code: string (nullable = true)
 |-- community: string (nullable = true)
 |-- community_code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- county: string (nullable = true)
 |-- county_code: string (nullable = true)
 |-- event_time: long (nullable = true)
 |-- geohash: string (nullable = true)
 |-- horizontal_accuracy: double (nullable = true)
 |-- idfa: string (nullable = true)
 |-- idfa_hash_alg: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- place: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- user_ip: string (nullable =

Humm that's a lot of columns, let's take a peek at our data. 

In [20]:
# show all columns
pd.set_option('display.max_columns', None)

In [21]:
df.limit(5).toPandas()

,action,api_key,app_id,beacon_major,beacon_minor,beacon_uuid,city,code,community,community_code,country_code,county,county_code,event_time,geohash,horizontal_accuracy,idfa,idfa_hash_alg,lat,lng,place,platform,state,state_code,user_ip
0,listrequest,9c4523f4cde116f32bf370656b35129cd9ced93c6ce6c1246,smartphones.collegescoreboard.football,0,0,,Littleton,80123,,,US,Jefferson,059,1491066280,9xj32w4r1z1m,0.0,c50ab245-d660-435e-a226-8d311d2aebbf,,39.628926,-105.092048,Littleton,android,Colorado,CO,184.96.241.157
1,virtual_beacon_list_request,9de1430e73cc96f15272a8a081d127746d3d4ef3d123b6adb,com.airkast.kbxxfm,0,0,,Houston,77067,,,US,Harris,201,1491066280,9vk47vts9tx0,9.0,8c14186b-070d-4933-a28f-a64e97afbb96,,29.957772,-95.452578,Houston,android,Texas,TX,45.18.76.214
2,listrequest,9c4523f4cde116f32bf370656b35129cd9ced93c6ce6c1246,smartphones.collegescoreboard.football,0,0,,Groton,01450,,,US,Middlesex,017,1491066280,drt4k62nrypf,0.0,1ed5c7d3-6b7b-4cbe-a726-7429eb1828c0,,42.596473,-71.531639,Groton,android,Massachusetts,MA,174.192.25.246
3,listrequest,e9d18b6c1b97cd776b1cd0c034a46738493151dc8e512acf0,com.checkout51.rc,0,0,,Media,19063,,,US,Delaware,045,1491066280,dr475f915hgy,0.0,9350ce13-d0a3-422b-a047-0f534d69c217,,39.916275,-75.419635,Media,ios,Pennsylvania,PA,107.77.204.113
4,listrequest,9c4523f4cde116f32bf370656b35129cd9ced93c6ce6c1246,smartphones.collegescoreboard.football,0,0,,Paw Paw,49079,,,US,Van Buren,159,1491066280,dp6zvrr3u6wk,0.0,a6e440f4-695b-42b2-a149-b6da243d7ea5,,42.183695,-85.891952,Paw Paw,android,Michigan,MI,50.107.82.231


Looking at this output, each row/record shows where a user/IDFA was located and at what time. Looks like the information is being generated by apps on a user's phone.      

I'm assuming beacon_major, beacon_minor and beacon_uuid have been deliberately masked.  

How many countries does this data cover? 

In [22]:
(df
 .select('country_code')
 .distinct()
 .show()
)

+------------+
|country_code|
+------------+
|          LT|
|          DZ|
|          FI|
|          PM|
|          RO|
|          NL|
|          PL|
|          RE|
|          MK|
|          MX|
|          GL|
|          SM|
|          AT|
|          RU|
|          HR|
|          LI|
|          CZ|
|          VA|
|          PT|
|          BD|
+------------+
only showing top 20 rows



In [23]:
(df
 .select('country_code')
 .distinct()
 .count()
)

69

humm that's a lot of countries ... i imagine some are more important than others ...

In [24]:
(df
#  .select('country_code')
#  .distinct()
 .groupby('country_code')
 .agg(func.count('*').alias('records'))
 .orderBy('records', ascending=False)
 .show()
)

+------------+-------+
|country_code|records|
+------------+-------+
|          US|6715156|
|          CA|1921178|
|          JP|  21594|
|          GU|  13946|
|          MX|  12011|
|          GB|   8618|
|          BR|   5658|
|          TR|   5634|
|          DE|   4259|
|          AU|   4217|
|          PK|   3687|
|          NL|   3093|
|          PH|   2904|
|          PR|   2794|
|          BE|   2619|
|          CO|   2421|
|          RU|   2378|
|          ES|   2106|
|          FR|   1768|
|          TH|   1763|
+------------+-------+
only showing top 20 rows



Not surprising, dataset is focused on USA and Canada.     
I like visualizing stuff so let's do a map of user activity.  

What time period does this data cover? Guessing a lot since its 50GB uncompressed ...

What is the action column all about?